<a href="https://colab.research.google.com/github/anushahulbatte/deeplearning/blob/main/DL_taska.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

if torch.__version__ != '2.5.1+cu124':
    !pip install torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cu124 -U --quiet
    print("PyTorch version updated to 2.5.1.")
else:
    print("PyTorch is already at the correct version (2.5.1).")

PyTorch is already at the correct version (2.5.1).


In [2]:
!pip install d2l==1.0.3 --quiet
!pip install scipy --quiet
!pip install torchmetrics --quiet

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import RandomSampler, random_split

from tqdm.auto import tqdm, trange
from matplotlib import pyplot as plt

DEVICE = torch.device("mps")

In [4]:
import random
import os
import matplotlib.pyplot as plt
import csv
import numpy as np


#from d2l import torch as d2l

Getting all the datasets required to work with

In [5]:
import os
! git clone https://git.wur.nl/bioinformatics/grs34806-deep-learning-project-data.git
os.chdir("grs34806-deep-learning-project-data")

fatal: destination path 'grs34806-deep-learning-project-data' already exists and is not an empty directory.


In [6]:
data = "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq"
datafile = open(data, "r")
print(datafile.read())

seq1	MQICGLVKTAKTPSCWSTVPYGREVGSPMTHQQHETERLQYKRWSGCNGFHGGKIRQYLFNMQGTDCKTDQTAGMAISESFLDFQGDHSYEELNAHPIFQTWPPNNSIRDDCL
seq2	MTGVKGTYGTPNVFQSTNEVSSGDQKIPWPKTEQTACLWGLPLSQFPIRKVIREIDNANTAIINLSLQDPNGNCNCMSKDSYTWWFRYARFHRCGRCIWMPWNGDRCDKYEKYYTDNMLYPEPNQLDSDRIKRGACHSDGMTDHGRDNGKWCCFVINLDDPSHTLP
seq3	MCHYAWCMWNTGCHWGINEEWAFHDKDLQGFPRKLHRPTRPKHYDTCLDDFSMCSTMCEARNVQRYNNWDRVRNRSQNVFDSYQDISRRAHFDSRQDDVLTDTKCYTNEFCHCELSPDMLFLGRTPNTSQAKPHIHVITRPNCNCHPRNQWATMIYYQWKLRQESETVIDQNKGCFSIFAKKEAGTWGIEFTLLYAK
seq4	MQVLNSMLDKVTQDWGKKCGLVKTAKTPQFWILVRRNPSVPVAAAMSINCFSTPRHELYKVRMMGWGSLYKNRQTYNSLSTLMRHSSMPSGVTDCVYWAVFGESIDDDWAWWLRADNDFLWFGNDTDRSIGLYINSCSRVDWMWANHFSE
seq5	MKNHQGCCPSTVGFRNDAQYQISWDVPMAWLWLDRPVKLYMFHFDTEMRDHFKIQSRVNQPDTYPVETTEPFAMVREPEAWSVTNRWSQNSRTWRNMCNTLIINNFEKYDANYVWRWQYWWERQGKRLPHWLPC
seq6	MNEWLADNWCMKQSSFGLPHKWILRNQHMWCLLWRWKICINHIEAWYYCKRAMDVTAQDSNKWEHKDDNKDGLYYVNTRIYAAQSWPHFVQFEAMWNPRVGSDQTYHDMLYMCIEDWEFPFIHVPISPFWVSNQASPDDMLWCIGMDCQKIPDTVHTPQMAENDTAHMYARINPRAEPVQMNMRD
seq7	MNLNWLMLGWPGGG

In [7]:
annotations = "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
annotationsfile = open(annotations, "r")
print(annotationsfile.read())

seq1
seq4
seq7
seq9
seq10
seq12
seq13
seq14
seq16
seq17
seq19
seq20
seq23
seq24
seq25
seq26
seq27
seq29
seq35
seq37
seq41
seq42
seq44
seq50
seq52
seq53
seq54
seq55
seq56
seq57
seq58
seq59
seq60
seq62
seq64
seq65
seq67
seq68
seq69
seq70
seq72
seq73
seq75
seq77
seq82
seq83
seq85
seq86
seq87
seq90
seq91
seq94
seq95
seq96
seq97
seq98
seq101
seq104
seq105
seq106
seq108
seq109
seq112
seq113
seq116
seq117
seq118
seq119
seq121
seq122
seq123
seq124
seq126
seq127
seq128
seq132
seq133
seq135
seq137
seq138
seq142
seq143
seq146
seq148
seq152
seq156
seq157
seq158
seq159
seq160
seq161
seq162
seq167
seq169
seq172
seq173
seq174
seq176
seq179
seq180
seq183
seq188
seq191
seq192
seq196
seq197
seq198
seq199
seq200
seq203
seq204
seq206
seq208
seq213
seq214
seq217
seq218
seq220
seq222
seq224
seq225
seq227
seq231
seq235
seq236
seq238
seq239
seq243
seq246
seq249
seq250
seq251
seq253
seq255
seq256
seq258
seq261
seq263
seq264
seq266
seq267
seq270
seq273
seq274
seq275
seq280
seq282
seq284
seq286
seq287
seq291
seq

In [8]:
def read(seqfile,posfile):

  """
  Extracting the sequences from seqfile. Making a label list in posfile.
  1's indicate sequences which have annotations and 0 indicate those that don't
  """
  # Remove the sequence identifiers
  with open(seqfile, 'r') as f:
    seq_lines = [seq_lines.strip() for seq_lines in f.readlines() if seq_lines.strip()]
    #print(seq_lines)
  seq_ids = []
  datalist = []
  current_seq = ""
  for line in seq_lines:
    if line.startswith("seq"):
        parts = line.split()
        seq_ids.append(parts[0])
        datalist.append(parts[1])

  with open(posfile, 'r') as f:
    annotated_ids = [line.strip() for line in f.readlines() if line.strip()]

  labellist = [1 if seq_id in annotated_ids else 0 for seq_id in seq_ids]

  assert len(datalist) == len(labellist) == len(seq_ids), "Lengths of lists don't match"

  print(f"Total sequences: {len(datalist)}")
  print(f"Length of the poslist: {len(labellist)}")
  print(f"Annotated sequences: {sum(labellist)}")
  print("\nFirst few entries:")
  for i in range(min(5, len(datalist))):
    print(f"ID: {seq_ids[i]}, Has annotation: {labellist[i]}, Sequence length: {len(datalist[i])}")

  return datalist, labellist


In [9]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134


Splitting the lists into training and testing dataset, to further use in the model.

In [10]:
import numpy
print(numpy.__version__)

1.23.5


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split

def generate_train_test(datalist, labellist):
  traindatalist, testdatalist, trainlabellist, testlabellist = train_test_split(
      datalist, labellist, test_size = 0.2, random_state=42
  )

  return traindatalist, trainlabellist, testdatalist, testlabellist


In [12]:
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)

# Printing the length of the training and testing data
print("Training data size:", len(traindatalist))
print("Training labels size:", len(trainlabellist))
print("Test data size:", len(testdatalist))
print("Test labels size:", len(testlabellist))

Training data size: 800
Training labels size: 800
Test data size: 200
Test labels size: 200


In [13]:
def tokenize(dat, map2num, non_aa_num=20):
  """
  This function takes in the sequences of AA. It uses dictionary mapping, to map
  AA to integers. The non_aa_num make up the unknown AA, and its default value is
  20.
  """
  seq = []
  for count, i in enumerate(dat):
      seq.append([map2num.get(j, non_aa_num) for j in list(i)])
  return seq

In [14]:
def build_seq_array(lines, num_steps, non_aa_num=20):
  """
  This function applies the truncate_pad function to all the sequences.
  It then converts the sequences into tensors.
  """
  array = torch.tensor([
      truncate_pad(l, num_steps, non_aa_num) for l in lines])
  return array

In [15]:
def truncate_pad(line, num_steps, padding_token):
  """
  The line in the parameters refers to the tokenized sequence.
  num_steps is the maxmimum sequence length. The padding_token is the integer
  for padding.
  The function pads shorter sequences with padding_token.
  """
  if len(line) > num_steps:
    return line[:num_steps] #Truncate
  return line + [padding_token] * (num_steps - len(line))

In [16]:
from d2l import torch as d2l

In [17]:
def load_data(batch_size, num_steps, dataset, trainlabels = None):
  mapaa2num = {aa: i for (i, aa)
                      in enumerate(list("ACDEFGHIKLMNPQRSTVWY"))}
  # Creating a dictionary which maps each AA to a unique integer
  seq, lab = dataset, trainlabels # lab is the labels which are associated with sequences

  seq = tokenize(seq, mapaa2num)
  seq_array = build_seq_array(seq, num_steps) # Shape = batch_size, num_steps

  # To convert the sequences to one-hot encoding
  # one_hot method takes a tensor and returns a tensor of shape of num_classes
  vocab_size = len(mapaa2num)
  seq_onehotencoding = F.one_hot(seq_array, num_classes=vocab_size).float()
  seq_onehotencoding = seq_onehotencoding.permute(0,2,1)
  # Shape = batch_size, vocab_size, num_steps

  data_arrays = (seq_onehotencoding, torch.tensor(lab))
  data_iter = d2l.load_array(data_arrays, batch_size)
  print("Shape of seq_array: ", seq_array.shape)
  print("Shape of data_arrays, which now considers one-hot encoded sequence: ",
        seq_onehotencoding.shape)
  return data_iter

In [18]:
# Example for 1 of the simulated datasets
dataset, labellist = read("/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
               "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos")
traindataset,traindatalabels, testdataset, testdatalabels = generate_train_test(dataset, labellist)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134


In [19]:
# Setting batch_size and num_steps (maximum sequence length)
train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])


In [20]:
class ProteinCNN1D(nn.Module):
  """
  Making a 1-D CNN for predicting if a given protein sequence performs the
  function and if the label corresponds to a specific GO ID.
  """
  #nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
  def __init__(self, vocab_size: int, context_size: int, conv_channels: int=128, use_bias: bool=False,
              momentum=0, dropout_rate: float=0.0):
    super().__init__()
    assert context_size % 2 == 0, f'Invalid block_size, {context_size} is not an even number'
    self.vocab_size = vocab_size
    self.context_size = context_size
    self.momentum = momentum

    self.cnn = nn.Sequential(
        nn.Conv1d(in_channels=vocab_size, #This is the 1-hot encoded AA sequences
                  out_channels = conv_channels,
                  kernel_size = 3,
                  padding='same',
                  bias=use_bias),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2, stride=2),
        nn.Dropout(dropout_rate),
        nn.Conv1d(
            in_channels=conv_channels,
            out_channels = conv_channels,
            kernel_size =3,
            padding='same',
            bias=use_bias
        ),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2, stride=2),
        nn.Dropout(dropout_rate),
        nn.Flatten(),
        nn.Dropout(dropout_rate/2),
        nn.Linear(
            in_features=int(conv_channels*context_size/4),
            out_features=1,
            bias=use_bias
        )
    )
  def forward(self, X:torch.tensor, targets: torch.tensor=None) -> tuple[torch.tensor, torch.tensor]:
    logits = self.cnn(X).squeeze(1) #Unsqueezing to remove additional dimension
    loss = None if targets is None else F.binary_cross_entropy_with_logits(logits, targets.float())
    return logits, loss



Training

In [21]:
DEVICE = torch.device("cpu")

In [22]:
def trainfunction(net, train_iter, test_iter, epochs=20, lr=0, weight_decay=0):
  optimizer = torch.optim.Adam(net.parameters(), lr=lr,
                               weight_decay=weight_decay)

  for epoch in range(epochs):
    net.train()
    total_loss = 0
    for X, y in train_iter:
      optimizer.zero_grad()
      logits, loss = net(X,y)
      loss.backward
      optimizer.step()
      total_loss += loss.item()

    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
      for X, y in test_iter:
        logits, _ = net(X)
        preds = (torch.sigmoid(logits) > 0.5).float()
        correct += (preds == y).sum().item()
        total += y.size(0)

    print(f"Epoch {epoch}, Loss: {total_loss/len(train_iter):.3f}, Test Acc: {correct/total:.3f}")



Training results with only LR as 0.001

In [23]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 15
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,  # 20 AAs + 1 padding token
    context_size=num_steps,
    conv_channels=128
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=0.001)


Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.693, Test Acc: 0.490
Epoch 1, Loss: 0.693, Test Acc: 0.490
Epoch 2, Loss: 0.693, Test Acc: 0.490
Epoch 3, Loss: 0.693, Test Acc: 0.490
Epoch 4, Loss: 0.693, Test Acc: 0.490
Epoch 5, Loss: 0.693, Test Acc: 0.490
Epoch 6, Loss: 0.693, Test Acc: 0.490
Epoch 7, Loss: 0.693, Test Acc: 0.490
Epoch 8, Loss: 0.693, Test Acc: 0.490
Epoch 9, Loss: 0.693, Test Acc: 0.490

Training results with lr as 0.001 and a dropout of 0.3

In [24]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 15
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,
    context_size=num_steps,
    conv_channels=128,
    dropout_rate=0.3
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=0.001, weight_decay=0)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.695, Test Acc: 0.450
Epoch 1, Loss: 0.696, Test Acc: 0.450
Epoch 2, Loss: 0.694, Test Acc: 0.450
Epoch 3, Loss: 0.693, Test Acc: 0.450
Epoch 4, Loss: 0.694, Test Acc: 0.450
Epoch 5, Loss: 0.692, Test Acc: 0.450
Epoch 6, Loss: 0.694, Test Acc: 0.450
Epoch 7, Loss: 0.694, Test Acc: 0.450
Epoch 8, Loss: 0.694, Test Acc: 0.450
Epoch 9, Loss: 0.694, Test Acc: 0.450

Training results with weight decay of 0.001 and no dropout to check effect of weight decay

In [25]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 15
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,
    context_size=num_steps,
    conv_channels=128
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=0.001, weight_decay=0.001)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.693, Test Acc: 0.500
Epoch 1, Loss: 0.693, Test Acc: 0.500
Epoch 2, Loss: 0.693, Test Acc: 0.500
Epoch 3, Loss: 0.693, Test Acc: 0.500
Epoch 4, Loss: 0.693, Test Acc: 0.500
Epoch 5, Loss: 0.693, Test Acc: 0.500
Epoch 6, Loss: 0.693, Test Acc: 0.500
Epoch 7, Loss: 0.693, Test Acc: 0.500
Epoch 8, Loss: 0.693, Test Acc: 0.500
Epoch 9, Loss: 0.693, Test Acc: 0.500

Training results with only momentum of 0.5





In [34]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 15
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,
    context_size=num_steps,
    conv_channels=128,
    momentum=0.5
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=0.001, weight_decay=0.01)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.728, Test Acc: 0.465
Epoch 1, Loss: 0.744, Test Acc: 0.460
Epoch 2, Loss: 0.738, Test Acc: 0.465
Epoch 3, Loss: 0.730, Test Acc: 0.465
Epoch 4, Loss: 0.737, Test Acc: 0.465
Epoch 5, Loss: 0.724, Test Acc: 0.465
Epoch 6, Loss: 0.729, Test Acc: 0.465
Epoch 7, Loss: 0.735, Test Acc: 0.465
Epoch 8, Loss: 0.730, Test Acc: 0.465
Epoch 9, Loss: 0.734, Test Acc: 0.465

Implementing multiple hyperparameters to optimize results

In [27]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=15, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=15, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 15
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,
    context_size=num_steps,
    conv_channels=128,
    momentum=0.55,
    dropout_rate=0.3
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=1e-4, weight_decay=0.0001)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.695, Test Acc: 0.480
Epoch 1, Loss: 0.695, Test Acc: 0.480
Epoch 2, Loss: 0.694, Test Acc: 0.480
Epoch 3, Loss: 0.695, Test Acc: 0.480
Epoch 4, Loss: 0.695, Test Acc: 0.480
Epoch 5, Loss: 0.693, Test Acc: 0.480
Epoch 6, Loss: 0.694, Test Acc: 0.480
Epoch 7, Loss: 0.694, Test Acc: 0.480
Epoch 8, Loss: 0.695, Test Acc: 0.480
Epoch 9, Loss: 0.694, Test Acc: 0.480

Trying with a different batch size:

In [28]:
def init_cnn(module):

    # Linear Layers
    if type(module) == nn.Linear:
        nn.init.kaiming_uniform_(module.weight, nonlinearity="sigmoid")

    # CNN Layers
    if type(module) == nn.Conv1d:
        nn.init.kaiming_uniform_(module.weight, nonlinearity="sigmoid")

In [29]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=30, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=30, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 30
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,  # 20 AAs + 1 padding token
    context_size=num_steps,
    conv_channels=128,
    momentum=0.55,
    dropout_rate=0.3
)

trainfunction(model, train_iter, test_iter, epochs=20, lr=1e-4,weight_decay=0.0001)


Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.693, Test Acc: 0.475
Epoch 1, Loss: 0.694, Test Acc: 0.475
Epoch 2, Loss: 0.694, Test Acc: 0.475
Epoch 3, Loss: 0.694, Test Acc: 0.475
Epoch 4, Loss: 0.694, Test Acc: 0.475
Epoch 5, Loss: 0.694, Test Acc: 0.475
Epoch 6, Loss: 0.693, Test Acc: 0.475
Epoch 7, Loss: 0.695, Test Acc: 0.475
Epoch 8, Loss: 0.694, Test Acc: 0.475
Epoch 9, Loss: 0.693, Test Acc: 0.475

Adding batch normalization to the architecture, and increasing the convolutions

In [30]:
class ProteinCNN1D(nn.Module):
  """
  Making a 1-D CNN for predicting if a given protein sequence performs the
  function and if the label corresponds to a specific GO ID.
  """
  #nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
  def __init__(self, vocab_size: int, context_size: int, conv_channels: int=128, use_bias: bool=False,
              momentum=0, dropout_rate: float=0.1):
    super().__init__()
    assert context_size % 2 == 0, f'Invalid block_size, {context_size} is not an even number'
    self.vocab_size = vocab_size
    self.context_size = context_size
    self.momentum = momentum

    self.cnn = nn.Sequential(
        nn.Conv1d(in_channels=vocab_size, #This is the 1-hot encoded AA sequences
                  out_channels = conv_channels,
                  kernel_size = 5, #Changing kernel size to 5 instead of 3
                  padding='same',
                  bias=use_bias),
        nn.BatchNorm1d(conv_channels),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2, stride=2),
        nn.Dropout(dropout_rate),

        nn.Conv1d(
            in_channels=conv_channels,
            out_channels = conv_channels,
            kernel_size =5,
            padding='same',
            bias=use_bias
        ),
        nn.BatchNorm1d(conv_channels),
        nn.ReLU(),
        nn.MaxPool1d(kernel_size=2, stride=2),
        nn.Dropout(dropout_rate),

        nn.Flatten(),
        nn.Dropout(dropout_rate/2),
        nn.Linear(
            in_features=int(conv_channels*context_size/4),
            out_features=1,
            bias=use_bias
        )
    )
  def forward(self, X:torch.tensor, targets: torch.tensor=None) -> tuple[torch.tensor, torch.tensor]:
    logits = self.cnn(X).squeeze(1) #Unsqueezing to remove additional dimension
    loss = None if targets is None else F.binary_cross_entropy_with_logits(logits, targets.float())
    return logits, loss


In [35]:
datalist, labellist = read(
seqfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.seq",
posfile =  "/content/grs34806-deep-learning-project-data/len100_200_n1000.pos"
)
traindatalist, trainlabellist, testdatalist, testlabellist = generate_train_test(datalist, labellist)


train_iter = load_data(batch_size=25, num_steps=20, dataset=traindataset, trainlabels=traindatalabels)
test_iter = load_data(batch_size=25, num_steps=20, dataset=testdataset, trainlabels =testdatalabels)

batch_size = 25
num_steps = 20

model = ProteinCNN1D(
    vocab_size=20,  # 20 AAs + 1 padding token
    context_size=num_steps,
    conv_channels=128,
    momentum=0.55,
    dropout_rate=0.3
)

trainfunction(model, train_iter, test_iter, epochs=50, lr=1e-4, weight_decay = 1e-3)

Total sequences: 1000
Length of the poslist: 1000
Annotated sequences: 507

First few entries:
ID: seq1, Has annotation: 1, Sequence length: 113
ID: seq2, Has annotation: 0, Sequence length: 166
ID: seq3, Has annotation: 0, Sequence length: 197
ID: seq4, Has annotation: 1, Sequence length: 150
ID: seq5, Has annotation: 0, Sequence length: 134
Shape of seq_array:  torch.Size([800, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([800, 20, 20])
Shape of seq_array:  torch.Size([200, 20])
Shape of data_arrays, which now considers one-hot encoded sequence:  torch.Size([200, 20, 20])
Epoch 0, Loss: 0.809, Test Acc: 0.530
Epoch 1, Loss: 0.805, Test Acc: 0.530
Epoch 2, Loss: 0.803, Test Acc: 0.535
Epoch 3, Loss: 0.770, Test Acc: 0.535
Epoch 4, Loss: 0.795, Test Acc: 0.535
Epoch 5, Loss: 0.800, Test Acc: 0.530
Epoch 6, Loss: 0.792, Test Acc: 0.535
Epoch 7, Loss: 0.810, Test Acc: 0.530
Epoch 8, Loss: 0.792, Test Acc: 0.530
Epoch 9, Loss: 0.791, Test Acc: 0.530